In [1]:
from classification_model import get_model

/home/jamesaustin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import yaml
from os import listdir
from os.path import isfile, join
import scipy
import matplotlib.pyplot as plt
import numpy as np
import random

def show_image(*imgs, title=''):
    if len(imgs) > 1:
        img = np.concatenate(tuple([x for x in imgs]), axis=1)
    else:
        img = imgs[0]
    fig = plt.figure(figsize=(10,100))
    plt.imshow(img)
    fig.suptitle(title)

def get_subimages(image, annotations):
    def get_annotation(annotation):
        xmin = int(annotation['xmin'])
        xmax = int((annotation['xmin'] + annotation['x_width']))
        ymin = int(annotation['ymin'])
        ymax = int((annotation['ymin'] + annotation['y_height']))
        return (image[ymin:ymax, xmin:xmax], annotation['class'])
    annotations.append({'xmin':image.shape[0] * random.uniform(0.4,0.6),
                        'ymin':image.shape[1] * random.uniform(0.4,0.6), 
                        'x_width':32, 'y_height':64, 'class': 'unknown'})
    return [get_annotation(x) for x in annotations]

def get_annotations(path):
    annotations = yaml.load(open(path, 'rb'))
    return {x['filename'].split('/')[1]: x['annotations'] for x in annotations}

def get_file(path, annotation):
    scale = 0.8
    base_image = scipy.misc.imread(path)
    subimages = get_subimages(base_image, annotation)
    return [(image, annotation) for image, annotation in subimages]
def get_sim_training_data():
    annotations = get_annotations('data/sim_training_data/sim_data_annotations.yaml')
    file_location = 'data/sim_training_data/sim_data_capture/'
    all_files = [(f, join(file_location, f)) for f in listdir(file_location) if isfile(join(file_location, f))]
    dataset = []
    for file_name, file_annotations in annotations.items():
        dataset.extend(get_file(join(file_location, file_name), file_annotations))
    
    def resize_image(image):
        #return image
        return scipy.misc.imresize(image, (64,32))
    dataset = [(resize_image(image), annotation) for image, annotation in dataset if min(image.shape) > 0]
    return dataset

dataset = get_sim_training_data()

/home/jamesaustin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/jamesaustin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:49: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import random
import scipy

dataset = shuffle(dataset)
training_set, test_set = train_test_split(dataset, test_size=0.3)
training_set, validation_set = train_test_split(training_set, test_size=0.1)

labels = {
    'Green': 0,
    'Yellow': 1,
    'Red': 2,
    'unknown': 3
}
reverse_labels = {y: x for x,y in labels.items()}

def myGenerator(dataset, permutate_amount=0.0):
    #loading data
    while True:
        for image, label in dataset:
            #if permutate_amount > random.random():
            #    image, label = permutate(image, label)
            yhotlabel = np.zeros((len(labels)))
            yhotlabel[labels[label]] = 1
            result = [np.array([image]), np.array([yhotlabel])]
            yield result
training_generator = myGenerator(training_set, permutate_amount=0.5)
validating_generator = myGenerator(validation_set)



In [4]:
from keras.callbacks import ModelCheckpoint, Callback
from keras_tqdm import TQDMNotebookCallback

model = get_model()

checkpointer = ModelCheckpoint(filepath="./tmp/classification_1_.{epoch:02d}-{val_loss:.2f}.hdf5", verbose=1)

model.fit_generator(training_generator, 
                    steps_per_epoch=len(training_set), 
                    epochs=20, 
                    verbose=0,
                    validation_data=validating_generator, 
                    validation_steps=len(validation_set), 
                    callbacks=[checkpointer,TQDMNotebookCallback()])

model.save_weights('./traffic_classification_model_v2.hdf5')

HBox(children=(IntProgress(value=0, description='Training', max=20), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 0', max=595), HTML(value='')))


Epoch 00001: saving model to ./tmp/classification_1_.01-0.48.hdf5


HBox(children=(IntProgress(value=0, description='Epoch 1', max=595), HTML(value='')))


Epoch 00002: saving model to ./tmp/classification_1_.02-0.81.hdf5


...


Epoch 00018: saving model to ./tmp/classification_1_.18-0.24.hdf5


HBox(children=(IntProgress(value=0, description='Epoch 18', max=595), HTML(value='')))


Epoch 00019: saving model to ./tmp/classification_1_.19-0.20.hdf5


HBox(children=(IntProgress(value=0, description='Epoch 19', max=595), HTML(value='')))


Epoch 00020: saving model to ./tmp/classification_1_.20-0.16.hdf5



In [6]:
for image, annotation in dataset:
    prediction = model.predict(np.array([image]))[0]
    correct_label = np.argmax(prediction)
    print('---')
    print(reverse_labels[correct_label])
    print(annotation)

---
unknown
unknown
---
Red
Red
---
Red
Red
---
unknown
unknown
---
Green
Green
---
Red
Red
---
Red
Red
---
Green
Green
---
Yellow
Yellow
---
unknown
unknown
---
unknown
unknown
---
Red
Red
---
Red
Red
---
Red
Red
---
unknown
unknown
---
Red
Red
---
Red
Red
---
Green
Green
---
unknown
unknown
---
Green
Green
---
unknown
unknown
---
unknown
unknown
---
Red
Red
---
Green
Green
---
unknown
unknown
---
Red
Red
---
unknown
unknown
---
Red
Red
---
Red
Red
---
unknown
unknown
---
unknown
unknown
---
unknown
unknown
---
unknown
unknown
---
Green
Green
---
Green
Green
---
Green
Green
---
Red
Red
---
unknown
unknown
---
Red
Red
---
Red
Red
---
Green
Green
---
unknown
unknown
---
Red
Red
---
unknown
unknown
---
Green
Green
---
Red
Red
---
Red
Red
---
Red
Red
---
Red
Red
---
Red
Red
---
Red
Red
---
Red
Red
---
Yellow
Yellow
---
unknown
unknown
---
Yellow
Yellow
---
Yellow
Yellow
---
unknown
Red
---
unknown
unknown
---
unknown
unknown
---
Red
Red
---
Red
Red
---
unknown
unknown
---
Yellow
Yellow
--

---
unknown
unknown
---
Red
Red
---
unknown
unknown
---
Red
Red
---
Red
Red
---
Red
Red
---
Yellow
Yellow
---
Green
Green
---
unknown
unknown
---
Red
Red
---
unknown
unknown
---
Red
Red
---
unknown
unknown
---
Red
Red
---
Red
Red
---
unknown
unknown
---
Yellow
Yellow
---
Red
Red
---
Red
Red
---
unknown
unknown
---
unknown
unknown
---
unknown
unknown
---
Red
Red
---
Red
Red
---
unknown
unknown
---
unknown
unknown
---
unknown
unknown
---
Green
Green
---
unknown
unknown
---
Yellow
Yellow
---
Red
Red
---
Red
Red
---
Red
Red
---
Yellow
Yellow
---
Red
Red
---
unknown
unknown
---
Green
Green
---
unknown
unknown
---
Red
Red
---
Red
Red
---
Yellow
Yellow
---
Red
Red
---
unknown
unknown
---
Yellow
Yellow
---
Green
Green
---
Red
Red
---
Red
Red
---
Red
Red
---
Green
Green
---
unknown
unknown
---
unknown
unknown
---
Red
Red
---
Red
Red
---
Red
Red
---
Red
Red
---
unknown
unknown
---
Red
Red
---
Red
Red
---
Red
Red
---
Red
Red
---
Red
Red
---
unknown
unknown
---
Red
Red
---
Red
Red
---
Red
Red
---
